<a href="https://colab.research.google.com/github/musah29/Teleco_Customer_Churn_Analysis/blob/main/LP3_TELCO_CHURN_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GROUP BEIJING**

**TELCOM CUSTOMER CHURN ANALYSIS**

In the highly competitive telecommunications industry, retaining customers is crucial for the success of any company. Customer churn, or the loss of customers to competitors or other reasons, can have a significant impact on a telecom company's revenue and profitability. Therefore, predicting and reducing customer churn is a top priority for many telecom companies. In this project, we aim to perform a churn analysis using four machine learning models to predict which customers are most likely to churn. By analyzing customer behavior, usage patterns, and other relevant factors, we can identify the key factors driving churn and develop targeted strategies to retain customers. The goal of this project is to provide valuable insights for telecom companies to reduce churn and improve customer retention rates.

IMPORTATIONS

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
customer_list=pd.read_csv('')

# New section